In [46]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import scipy.linalg as linalg # Import the dense linear algebra module



In [3]:

# Load node features and labels
column_names = ['node_id'] + [f'w_{i}' for i in range(1433)] + ['subject']
node_data = pd.read_csv('cora.content', sep='\t', header=None, names=column_names)

# Load edges
edge_data = pd.read_csv('cora.cites', sep='\t', header=None, names=['target', 'source'])

# Create a directed graph
g= nx.from_pandas_edgelist(edge_data, source='source', target='target', create_using=nx.DiGraph())


# Step1: Preprocessing/Train-Test Split


In [37]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        return [to_tuple(mx) for mx in sparse_mx]
    else:
        return to_tuple(sparse_mx)


def mask_test_edges_directed(adj, test_frac=.05, val_frac=.05,
    prevent_disconnect=False, verbose=False, false_edge_sampling='random'):

    if verbose:
        print('preprocessing...')

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    # Convert to networkx graph to calc num. weakly connected components
    # Create graph from the adjacency matrix
    # Use from_scipy_sparse_array instead of the deprecated from_scipy_sparse_matrix
    g = nx.from_scipy_sparse_array(adj, create_using=nx.DiGraph())
    orig_num_wcc = nx.number_weakly_connected_components(g)

    adj_tuple = sparse_to_tuple(adj) # (coords, values, shape)
    edges = adj_tuple[0] # List of ALL edges (either direction)
    # Store edges as list of tuples (from_node, to_node)
    edge_pairs = [(edge[0], edge[1]) for edge in edges]

    num_test = int(np.floor(edges.shape[0] * test_frac)) # controls how large the test set should be
    num_val = int(np.floor(edges.shape[0] * val_frac)) # controls how large the validation set should be
    num_train = len(edge_pairs) - num_test - num_val # num train edges

    all_edge_set = set(edge_pairs)
    train_edges = set(edge_pairs) # init train_edges to have all edges
    test_edges = set() # init test_edges as empty set
    val_edges = set() # init val edges as empty set

    ### ---------- TRUE EDGES ---------- ###
    # Shuffle and iterate over all edges
    np.random.shuffle(edge_pairs)

    # get initial bridge edges from the undirected version of the graph
    bridge_edges = set(nx.bridges(nx.to_undirected(g)))

    if verbose:
        print('creating true edges...')

    # Create a copy of the graph to remove edges from temporarily
    g_temp = g.copy()

    for ind, edge in enumerate(edge_pairs):
        node1, node2 = edge[0], edge[1]

        # Recalculate bridges every ____ iterations to be relatively recent
        if ind % 10000 == 0:
            bridge_edges = set(nx.bridges(nx.to_undirected(g_temp)))

        # Don't sample bridge edges to increase likelihood of staying connected
        # Check for both directions in undirected bridges
        if (node1, node2) in bridge_edges or (node2, node1) in bridge_edges:
            continue

        # If removing edge would disconnect the graph, backtrack and move on
        g_temp.remove_edge(node1, node2)
        if prevent_disconnect:
            if not nx.is_weakly_connected(g_temp):
                g_temp.add_edge(node1, node2) # Add back the edge if it disconnects
                continue

        # Fill test_edges first
        if len(test_edges) < num_test:
            test_edges.add(edge)
            # Remove from train_edges set for true edges
            if edge in train_edges:
                train_edges.remove(edge)
            if verbose and len(test_edges) % 10000 == 0:
                print('Current num test edges: ', len(test_edges))

        # Then, fill val_edges
        elif len(val_edges) < num_val:
            val_edges.add(edge)
            # Remove from train_edges set for true edges
            if edge in train_edges:
                train_edges.remove(edge)
            if verbose and len(val_edges) % 10000 == 0:
                print('Current num val edges: ', len(val_edges))

        # Both edge lists full --> break loop
        elif len(test_edges) == num_test and len(val_edges) == num_val:
            break

    # Check that enough test/val edges were found
    if (len(val_edges) < num_val or len(test_edges) < num_test):
        print(f"WARNING: not enough removable edges to perform full train-test split! Requested: Test {num_test}, Val {num_val}. Returned: Test {len(test_edges)}, Val {len(val_edges)}")


    # Calculate largest weakly connected component on the graph after removing test/val true edges
    wccs = list(nx.weakly_connected_components(g_temp))
    if not wccs:
         print("WARNING: Graph became disconnected after removing edges and has no weakly connected components.")
         largest_wcc_set = set()
         largest_wcc = nx.DiGraph()
    else:
        largest_wcc_set = max(wccs, key=len)
        largest_wcc = g_temp.subgraph(largest_wcc_set)


    # Print stats for largest remaining WCC
    print('Num WCC in remaining graph: ', nx.number_weakly_connected_components(g_temp))
    print('Largest WCC num nodes: ', largest_wcc.number_of_nodes())
    print('Largest WCC num edges: ', largest_wcc.number_of_edges())

    if prevent_disconnect:
         # Recompute WCC on the graph with train edges only
         g_train_check = nx.from_edgelist(list(train_edges), create_using=nx.DiGraph())
         # Check if the number of weakly connected components is the same as the original graph
         # Handle the case where the original graph might have had multiple components and
         # the largest WCC filtering removes some nodes entirely.
         # A stricter check could be that the nodes in g_train_check are a subset of original nodes
         # and WCC count is the same for the induced subgraph on original nodes
         # For now, just check if the main component is still connected like the original
         # This requires careful consideration based on the desired behavior.
         # If the goal is to work *only* within the largest WCC of the original graph,
         # the orig_num_wcc should be based on the largest WCC of the *original* graph.
         # If the goal is to maintain connectivity of the *entire* original graph structure,
         # this assertion needs refinement.
         # Assuming the goal is to maintain connectivity of the largest component:
         g_orig_largest_wcc = g.subgraph(max(nx.weakly_connected_components(g), key=len))
         # Only perform the connectivity check if both the original largest WCC and the training graph are not empty
         #if g_orig_largest_wcc.number_of_nodes() > 0 and g_train_check.number_of_nodes() > 0:
             # Need to ensure the nodes of g_orig_largest_wcc are present in g_train_check before creating subgraph
          #   nodes_in_g_train_check = set(g_train_check.nodes())
           #  nodes_in_orig_wcc_and_train = sorted(list(g_orig_largest_wcc.nodes() & nodes_in_g_train_check))
            # if nodes_in_orig_wcc_and_train: # Check if there are common nodes
             #    assert nx.is_weakly_connected(g_train_check.subgraph(nodes_in_orig_wcc_and_train)) == nx.is_weakly_connected(g_orig_largest_wcc.subgraph(nodes_in_orig_wcc_and_train))
             #else:
             #    # If no common nodes, a strict connectivity assertion might not be meaningful here
              #   print("INFO: No common nodes between original largest WCC and training graph. Skipping connectivity assertion.")


    # Filter edges to only include those where both endpoints are in the largest WCC
    # This is crucial for ensuring the downstream tasks operate within a connected component
    train_edges = {edge for edge in train_edges if edge[0] in largest_wcc_set and edge[1] in largest_wcc_set}
    test_edges = {edge for edge in test_edges if edge[0] in largest_wcc_set and edge[1] in largest_wcc_set}
    val_edges = {edge for edge in val_edges if edge[0] in largest_wcc_set and edge[1] in largest_wcc_set}


    ### ---------- FALSE EDGES ---------- ###

    # Initialize empty sets
    train_edges_false = set()
    test_edges_false = set()
    val_edges_false = set()

    # Generate candidate false edges (from g-complement) and iterate through them
    if false_edge_sampling == 'iterative':
        if verbose:
            print("preparing complement adjacency matrix...")

        # Sample false edges from G-complement, instead of randomly generating edges
        # Create a sparse adjacency matrix for the graph with training edges
        nodes_in_wcc_list = sorted(list(largest_wcc_set))
        num_nodes_wcc = len(nodes_in_wcc_list)

        if num_nodes_wcc == 0:
            print("WARNING: Largest WCC is empty. Cannot generate false edges.")
        else:
            # Create adj matrix for the largest WCC subgraph
            # Use the nodes from the largest WCC found *after* removing test/val true edges
            adj_wcc = nx.adjacency_matrix(largest_wcc, nodelist=nodes_in_wcc_list).astype(np.int8)

            # Find coordinates of zero entries (non-existing edges) within the WCC subgraph's adjacency matrix
            adj_wcc_coo = adj_wcc.tocoo()
            existing_coords_wcc_indices = set((r, c) for r, c in zip(adj_wcc_coo.row, adj_wcc_coo.col))

            edge_pairs_false_candidates = []
            for i in range(num_nodes_wcc):
                for j in range(num_nodes_wcc):
                    if i != j and (i, j) not in existing_coords_wcc_indices:
                        # Map WCC indices back to original node IDs
                        original_u = nodes_in_wcc_list[i]
                        original_v = nodes_in_wcc_list[j]
                        edge_pairs_false_candidates.append((original_u, original_v))

            # Shuffle and iterate over false edges
            np.random.shuffle(edge_pairs_false_candidates)
            if verbose:
                print("adding candidate false edges to false edge sets...")

            for false_edge in edge_pairs_false_candidates:
                 # Ensure the false edge is between nodes within the largest WCC (already done by candidate generation)
                 # Ensure it's not an existing edge (should be guaranteed by construction)
                 # No need to check all_edge_set explicitly since candidates are from complement within WCC

                 # Fill train_edges_false first
                 if len(train_edges_false) < len(train_edges):
                    train_edges_false.add(false_edge)
                    if verbose and len(train_edges_false) % 100000 == 0:
                        print(f'Current num false train edges: {len(train_edges_false)}')

                 # Fill test_edges_false next
                 elif len(test_edges_false) < len(test_edges):
                    # Ensure it's not already in train/val false
                    if false_edge not in train_edges_false and false_edge not in val_edges_false:
                       test_edges_false.add(false_edge)
                       if verbose and len(test_edges_false) % 100000 == 0:
                           print(f'Current num false test edges: {len(test_edges_false)}')

                 # Fill val_edges_false last
                 elif len(val_edges_false) < len(val_edges):
                    # Ensure it's not already in train/test false
                    if false_edge not in train_edges_false and false_edge not in test_edges_false:
                       val_edges_false.add(false_edge)
                       if verbose and len(val_edges_false) % 100000 == 0:
                           print(f'Current num false val edges: {len(val_edges_false)}')

                 # All sets filled --> break
                 elif len(train_edges_false) == len(train_edges) and \
                      len(test_edges_false) == len(test_edges) and \
                      len(val_edges_false) == len(val_edges):
                     break

            # If not enough false edges were found in the iterative approach, warn the user
            if (len(train_edges_false) < len(train_edges) or
                len(test_edges_false) < len(test_edges) or
                len(val_edges_false) < len(val_edges)):
                 print("WARNING: Not enough false edge candidates found in iterative sampling within the largest WCC to match the number of true edges in each set.")
                 print(f"Required false edges: Train {len(train_edges)}, Test {len(test_edges)}, Val {len(val_edges)}")
                 print(f"Returned false edges: Train {len(train_edges_false)}, Test {len(test_edges_false)}, Val {len(val_edges_false)}")


    # Randomly generate false edges (idx_i, idx_j) 1 at a time to save memory
    elif false_edge_sampling == 'random':
        if verbose:
            print('creating false edges...')

        num_nodes = adj.shape[0]
        largest_wcc_list = list(largest_wcc_set) # Convert set to list for random sampling

        # FALSE TEST EDGES
        if verbose:
            print('creating false test edges...')
        while len(test_edges_false) < len(test_edges):
            if not largest_wcc_list:
                 print("WARNING: Largest WCC is empty. Cannot generate false edges.")
                 break

            # Sample nodes only from the largest WCC
            # Ensure replace=True is used as nodes can be sampled multiple times before forming a valid false edge
            idx_i, idx_j = np.random.choice(largest_wcc_list, 2, replace=True)


            if idx_i == idx_j: # no self-loops
                continue

            false_edge = (idx_i, idx_j)

            # Make sure false_edge not an actual edge, and not a repeat in any false set
            if false_edge in all_edge_set or \
               false_edge in test_edges_false or \
               false_edge in val_edges_false or \
               false_edge in train_edges_false:
                continue

            test_edges_false.add(false_edge)

            if verbose and len(test_edges_false) % 100000 == 0:
                print(f'Current num false test edges: {len(test_edges_false)}')

        # FALSE VAL EDGES
        if verbose:
            print('creating false val edges...')

        while len(val_edges_false) < len(val_edges):
            if not largest_wcc_list:
                 break
            # Sample nodes only from the largest WCC
            idx_i, idx_j = np.random.choice(largest_wcc_list, 2, replace=True)
            if idx_i == idx_j:
                continue

            false_edge = (idx_i, idx_j)

            # Make sure false_edge in not an actual edge, not in test_edges_false, not a repeat
            if false_edge in all_edge_set or \
                false_edge in test_edges_false or \
                false_edge in val_edges_false or \
                false_edge in train_edges_false:
                continue

            val_edges_false.add(false_edge)

            if verbose and len(val_edges_false) % 100000 == 0:
                print(f'Current num false val edges: {len(val_edges_false)}')

        # FALSE TRAIN EDGES
        if verbose:
            print('creating false train edges...')

        while len(train_edges_false) < len(train_edges):
            if not largest_wcc_list:
                 break
            # Sample nodes only from the largest WCC
            idx_i, idx_j = np.random.choice(largest_wcc_list, 2, replace=True)
            if idx_i == idx_j:
                continue

            false_edge = (idx_i, idx_j)

            # Make sure false_edge in not an actual edge, not in test_edges_false,
                # not in val_edges_false, not a repeat
            if false_edge in all_edge_set or \
                false_edge in test_edges_false or \
                false_edge in val_edges_false or \
                false_edge in train_edges_false:
                continue

            train_edges_false.add(false_edge)

            if verbose and len(train_edges_false) % 100000 == 0:
                print(f'Current num false train edges: {len(train_edges_false)}')


    ### ---------- FINAL DISJOINTNESS CHECKS ---------- ###
    if verbose:
        print('final checks for disjointness...')

    # assert: false_edges are actually false (not in all_edge_tuples)
    assert test_edges_false.isdisjoint(all_edge_set)
    assert val_edges_false.isdisjoint(all_edge_set)
    assert train_edges_false.isdisjoint(all_edge_set)

    # assert: test, val, train false edges disjoint
    assert test_edges_false.isdisjoint(val_edges_false)
    assert test_edges_false.isdisjoint(train_edges_false)
    assert val_edges_false.isdisjoint(train_edges_false)

    # assert: test, val, train positive edges disjoint
    assert val_edges.isdisjoint(train_edges)
    assert test_edges.isdisjoint(train_edges)
    assert val_edges.isdisjoint(test_edges)

    if verbose:
        print('creating adj_train...')

    # Re-build adj matrix using remaining graph (train edges only)
    # Create a graph with only the training edges
    g_train = nx.from_edgelist(list(train_edges), create_using=nx.DiGraph())

    # Ensure the adjacency matrix has the same shape as the training graph based on the nodes it actually contains
    # Use from_scipy_sparse_array
    # Use the nodes from the training graph itself for the nodelist
    adj_train = nx.adjacency_matrix(g_train, nodelist=sorted(list(g_train.nodes()))).astype(np.int8)


    # Convert edge-lists to numpy arrays
    # Sort the sets first to ensure consistent ordering for numpy array conversion
    train_edges = np.array(sorted(list(train_edges)))
    train_edges_false = np.array(sorted(list(train_edges_false)))
    val_edges = np.array(sorted(list(val_edges)))
    val_edges_false = np.array(sorted(list(val_edges_false)))
    test_edges = np.array(sorted(list(test_edges)))
    test_edges_false = np.array(sorted(list(test_edges_false)))

    if verbose:
        print('Done with train-test split!')
        print(f'Num train edges (true, false): ({train_edges.shape[0]}, {train_edges_false.shape[0]})')
        print(f'Num test edges (true, false): ({test_edges.shape[0]}, {test_edges_false.shape[0]})')
        print(f'Num val edges (true, false): ({val_edges.shape[0]}, {val_edges_false.shape[0]})')
        print('')

    # Return final edge lists (edges can go either direction!) and the training adjacency matrix
    return adj_train, train_edges, train_edges_false, \
        val_edges, val_edges_false, test_edges, test_edges_false




In [38]:
# Create a sparse adjacency matrix from the graph for the function
adj = nx.adjacency_matrix(g).astype(np.int8)

# Call the function to perform the split
adj_train, train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_directed(adj, verbose=True)

print("Train True Edges shape:", train_edges.shape)
print("Train False Edges shape:", train_edges_false.shape)
print("Validation True Edges shape:", val_edges.shape)
print("Validation False Edges shape:", val_edges_false.shape)
print("Test True Edges shape:", test_edges.shape)
print("Test False Edges shape:", test_edges_false.shape)

preprocessing...
creating true edges...
Num WCC in remaining graph:  88
Largest WCC num nodes:  2470
Largest WCC num edges:  4676
creating false edges...
creating false test edges...
creating false val edges...
creating false train edges...
final checks for disjointness...
creating adj_train...
Done with train-test split!
Num train edges (true, false): (4677, 4677)
Num test edges (true, false): (252, 252)
Num val edges (true, false): (251, 251)

Train True Edges shape: (4677, 2)
Train False Edges shape: (4677, 2)
Validation True Edges shape: (251, 2)
Validation False Edges shape: (251, 2)
Test True Edges shape: (252, 2)
Test False Edges shape: (252, 2)


In [39]:
g_train = nx.from_scipy_sparse_array(adj_train) # new graph object with only non-hidden edges


In [29]:
def get_roc_score(edges_pos, edges_neg, score_matrix):
    # Store positive edge predictions, actual values
    preds_pos = []
    pos = []
    for edge in edges_pos:
        preds_pos.append(score_matrix[edge[0], edge[1]]) # predicted score
        pos.append(adj[edge[0], edge[1]]) # actual value (1 for positive)

    # Store negative edge predictions, actual values
    preds_neg = []
    neg = []
    for edge in edges_neg:
        preds_neg.append(score_matrix[edge[0], edge[1]]) # predicted score
        neg.append(adj[edge[0], edge[1]]) # actual value (0 for negative)

    # Calculate scores
    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    return roc_score, ap_score

# Adamic-Adar


In [30]:
# Compute Adamic-Adar indexes from g_train
aa_matrix = np.zeros(adj.shape)
for u, v, p in nx.adamic_adar_index(g_train): # (u, v) = node indices, p = Adamic-Adar index
    aa_matrix[u][v] = p
    aa_matrix[v][u] = p # make sure it's symmetric

# Normalize array
aa_matrix = aa_matrix / aa_matrix.max()

In [31]:
# Calculate ROC AUC and Average Precision
aa_roc, aa_ap = get_roc_score(test_edges, test_edges_false, aa_matrix)

print('Adamic-Adar Test ROC score: ', str(aa_roc))
print('Adamic-Adar Test AP score: ', str(aa_ap))

Adamic-Adar Test ROC score:  0.5997314453125
Adamic-Adar Test AP score:  0.5982874517097334


#Jaccard Coefficient


In [32]:
# Compute Jaccard Coefficients from g_train
jc_matrix = np.zeros(adj.shape)
for u, v, p in nx.jaccard_coefficient(g_train): # (u, v) = node indices, p = Jaccard coefficient
    jc_matrix[u][v] = p
    jc_matrix[v][u] = p # make sure it's symmetric

# Normalize array
jc_matrix = jc_matrix / jc_matrix.max()

In [34]:
# Calculate ROC AUC and Average Precision
jc_roc, jc_ap = get_roc_score(test_edges, test_edges_false, jc_matrix)

print('Jaccard Coefficient Test ROC score: ', str(jc_roc))
print('Jaccard Coefficient Test AP score: ', str(jc_ap))

Jaccard Coefficient Test ROC score:  0.6005935668945312
Jaccard Coefficient Test AP score:  0.6036074501509878


#Preferential Attachment


In [35]:
# Calculate, store Adamic-Index scores in array
pa_matrix = np.zeros(adj.shape)
for u, v, p in nx.preferential_attachment(g_train): # (u, v) = node indices, p = Jaccard coefficient
    pa_matrix[u][v] = p
    pa_matrix[v][u] = p # make sure it's symmetric

# Normalize array
pa_matrix = pa_matrix / pa_matrix.max()

In [36]:
# Calculate ROC AUC and Average Precision
pa_roc, pa_ap = get_roc_score(test_edges, test_edges_false, pa_matrix)

print('Preferential Attachment Test ROC score: ', str(pa_roc))
print('Preferential Attachment Test AP score: ', str(pa_ap))

Preferential Attachment Test ROC score:  0.5609359741210938
Preferential Attachment Test AP score:  0.5763347205018841


#Common Neighbors

In [44]:
# Calculate, store Common Neighbor scores in array
cn_matrix = np.zeros(adj.shape)
# Iterate over all pairs of nodes in g_train and their common neighbors count
for u, v, p in nx.common_neighbors(g_train,u,v): # (u, v) = node indices, p = num_common_neighbors
    cn_matrix[u][v] = p
    cn_matrix[v][u] = p # make sure it's symmetric (based on undirected interpretation)

# Normalize array
max_cn = cn_matrix.max()
if max_cn > 0:
    cn_matrix = cn_matrix / max_cn
else:
    # If max_cn is 0, the matrix remains zeros, which is the correct normalized form in this case.
    pass

#cn_matrix = cn_matrix / cn_matrix.max()

# Calculate ROC AUC and Average Precision
cn_roc, cn_ap = get_roc_score(test_edges, test_edges_false, cn_matrix)

print('Common Neighbors Test ROC score: ', str(cn_roc))
print('Common Neighbors Test AP score: ', str(cn_ap))

Common Neighbors Test ROC score:  0.5
Common Neighbors Test AP score:  0.5
